In [25]:
# read in UK_flattened_output_big_eps.npy
import numpy as np
import matplotlib.pyplot as plt
#
file_names = ['UK_finer_feb2.npy','US_finer_feb2.npy']

Parameters used

In [31]:
epsilon = 0.01
infty = 10000000
state_values = np.arange(-2.8,2.8,0.28)
S = np.arange(0,20,1)
costs = np.arange(0,3,0.6) # change these params depending on what I used in running the notebook
possible_soc_terms = np.arange(0,3,0.6)
possible_inf_terms = np.arange(0,3,0.6)
theta_to_test = np.arange(0,20,2)

Reshaping when for looping over t1,t2,t3

In [28]:
all_results = []
for name in file_names:
    print(name)
    results = np.load(name, allow_pickle=True)
    # results is a flattened array, and I want to reshape it so results[v0,v1,v2,v3,v4,v5,v6,v7,v8] is the same as logloss(v0,v1,v2,v3,v4,v5,v6,v7,v8)
    results = results.reshape(len(theta_to_test),len(theta_to_test),len(theta_to_test),len(possible_soc_terms),len(possible_inf_terms),len(costs),len(theta_to_test),len(theta_to_test),len(theta_to_test)).transpose(0,1,2,8,7,6,5,4,3)
    # but I can't keep the whole array as it uses too much memory, so I will first identify which thetas are good
    best_thetas = np.zeros((len(costs),len(possible_inf_terms),len(possible_soc_terms)),dtype=int)
    for c in range(len(costs)):
        for inf in range(len(possible_inf_terms)):
            for soc in range(len(possible_soc_terms)):
                # best thetas[c,inf,soc] is the index of the best theta for specific cost, inf and soc combination
                best_thetas[c,inf,soc] = np.argmax(results[:,:,:,:,:,:,c,inf,soc])
                # print(costs[c],possible_inf_terms[inf],possible_soc_terms[soc],best_thetas[c,inf,soc],np.max(results[:,:,:,:,:,:,c,inf,soc]))
    unique_thetas = np.unique(best_thetas)
    cube_for_each_thetas = [results.flatten()[index*125:(index+1)*125] for index in unique_thetas ] # this stores the flattened version of results[t0,t1,t2,t3,t4,t5,:,:,:]
    # each element of cube_for_each_thetas is an array of len 125
    unique_thetas = [convert_to_thetas(theta) for theta in unique_thetas]
    del results
    all_results.append((cube_for_each_thetas ,unique_thetas,best_thetas))

UK_finer_feb2.npy
(125000000,)
0.0 0.0 0.0 0 -2021.459
0.0 0.0 0.1 0 -2021.4589
0.0 0.0 0.2 0 -2021.4589
0.0 0.0 0.30000000000000004 0 -2021.459
0.0 0.0 0.4 0 -2021.459
0.0 0.25 0.0 493355 -1862.9369
0.0 0.25 0.1 403355 -1859.2175
0.0 0.25 0.2 403355 -1858.6974
0.0 0.25 0.30000000000000004 403355 -1859.6208
0.0 0.25 0.4 403355 -1861.2649
0.0 0.5 0.0 402255 -1919.5648
0.0 0.5 0.1 402255 -1916.6819
0.0 0.5 0.2 402255 -1916.3379
0.0 0.5 0.30000000000000004 402255 -1916.5778
0.0 0.5 0.4 402255 -1917.1545
0.0 0.75 0.0 900241 -1969.2621
0.0 0.75 0.1 900241 -1968.1165
0.0 0.75 0.2 900241 -1967.637
0.0 0.75 0.30000000000000004 900241 -1967.5339
0.0 0.75 0.4 900241 -1967.7812
0.0 1.0 0.0 900241 -1978.7617
0.0 1.0 0.1 900241 -1978.5613
0.0 1.0 0.2 900241 -1978.9286
0.0 1.0 0.30000000000000004 900241 -1979.6855
0.0 1.0 0.4 900241 -1980.8429
0.6 0.0 0.0 0 -2021.4589
0.6 0.0 0.1 0 -2021.459
0.6 0.0 0.2 0 -2021.459
0.6 0.0 0.30000000000000004 0 -2021.459
0.6 0.0 0.4 0 -2021.4589
0.6 0.25 0.0 403355 

In [29]:
# convert to a flattened arr of length 125 to a 3d array
for i in range(len(all_results)):
    cubes = all_results[i][0]
    for j in range(len(cubes)):
        all_results[i][0][j] = cubes[j].reshape(5,5,5)

Plot graphs for each of the best thetas

In [119]:
def plot_results(cube_for_each_thetas,unique_thetas, cost_pos,inf_pos,soc_pos,country):
    fig, axes = plt.subplots(4, 4, figsize=(15, 15))
    cmap = plt.cm.viridis.copy()  
    cmap.set_under('black') 
    axes = axes.flatten()
    if cost_pos == 0:
        x_var = 'cost'
    if inf_pos == 0:
        x_var = 'inf'
    if soc_pos == 0:
        x_var = 'soc'
    if cost_pos == 1:
        y_var = 'cost'
    if inf_pos == 1:
        y_var = 'inf'
    if soc_pos == 1:
        y_var = 'soc'
    for i, cube in enumerate(cube_for_each_thetas):
        grid = np.zeros((5, 5))
        for y_index in range(5):
            for x_index in range(5):    
                grid[y_index, x_index] = np.max(cube.transpose(cost_pos,inf_pos,soc_pos)[x_index,y_index,:])
        ax = axes[i]
        max_val = np.max(grid)
        im = ax.imshow(grid, cmap=cmap,vmin=max_val-20)
        # label x and y axes
        ax.set_xlabel(x_var)
        ax.set_ylabel(y_var)
        ax.set_title(f'Thetas {unique_thetas[i]}')
        fig.colorbar(im, ax=ax)
    plt.suptitle(f'{y_var} against {x_var} for {country}')
    plt.tight_layout()
    plt.show()

In [ ]:
var_to_position = {'x':0,'y':1,'collapse':2}
for i, (cube_for_each_thetas ,unique_thetas,best_thetas) in enumerate(all_results[:]):
    print(file_names[i])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['x'], var_to_position['y'],var_to_position['collapse'],file_names[i])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['x'], var_to_position['collapse'],var_to_position['y'],file_names[i])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['collapse'], var_to_position['y'],var_to_position['x'],file_names[i])

Printing best logloss for each of best thetas

In [ ]:
# define functions to convert actual param values to their indices
def convert_to_thetas(theta_index):
    theta_index = (int(theta_index*10**-5),int((theta_index%10**5)*10**-4),int((theta_index%10**4)*10**-3),int((theta_index%10**3)*10**-2),int((theta_index%10**2)*10**-1),int(theta_index%10))
    return tuple(float(round(theta_to_test[theta],3) )for theta in theta_index)
def convert_coeffs(flat_index):
    return (int(flat_index/25),int((flat_index%25)/5),int((flat_index%5)))

In [37]:
for i, (cube_for_each_thetas ,unique_thetas,best_thetas) in enumerate(all_results):
    print(file_names[i])
    print("Best logloss for this file:",np.max([cube_for_each_thetas[i] for i in range(len(cube_for_each_thetas))]))   
    for i in range(len(cube_for_each_thetas)):
        print(f"Best logloss for the theta terms {unique_thetas[i]}:    logloss ={np.max(cube_for_each_thetas[i])} cost = {costs[(convert_coeffs(np.argmax(cube_for_each_thetas[i]))[0])]}, inf_term = {possible_inf_terms[(convert_coeffs(np.argmax(cube_for_each_thetas[i]))[1])]}, soc_term = {possible_soc_terms[(convert_coeffs(np.argmax(cube_for_each_thetas[i]))[2])]}" )

UK_finer_feb2.npy
Best logloss for this file: -1855.2284
Best logloss for the theta terms (-2.8, -2.8, -2.8, -2.8, -2.8, -2.8):    logloss =-2021.4588623046875 cost = 0.0, inf_term = 0.0, soc_term = 0.6
Best logloss for the theta terms (-1.12, -2.8, -1.12, -1.12, 0.0, 0.0):    logloss =-1864.49267578125 cost = 0.6, inf_term = 0.6, soc_term = 1.7999999999999998
Best logloss for the theta terms (-0.56, -2.8, -1.68, -1.68, 0.0, 0.0):    logloss =-1862.4921875 cost = 1.2, inf_term = 0.6, soc_term = 0.0
Best logloss for the theta terms (-0.56, -2.8, -1.12, -1.12, 0.0, 0.0):    logloss =-1855.2283935546875 cost = 1.2, inf_term = 0.6, soc_term = 0.6
Best logloss for the theta terms (-0.56, -2.24, -1.68, -1.68, 0.0, 0.0):    logloss =-1865.610107421875 cost = 1.2, inf_term = 0.6, soc_term = 0.0
Best logloss for the theta terms (-0.56, 2.24, -1.12, -1.12, 0.0, 0.0):    logloss =-1861.92041015625 cost = 0.6, inf_term = 0.6, soc_term = 0.0
Best logloss for the theta terms (2.24, -2.8, -2.8, -1.68

In [ ]:
logloss = (-1855.2284), (cost = 1.2, inf_term = 0.6, soc_term = 0), thetas = (-0.56, -2.8, -1.12, -1.12, 0.0, 0.0)
logloss = (-1888.0813), (cost = 1, inf_term = 1, soc_term = 0), (-0.56, -2.8, -1.68, -1.68, 0.0, 0.0)
logloss = (-2237.1602), (cost = 1.2, inf_term = 0.6, soc_term = 0), (-1.12, -2.8, -1.12, -0.56, -0.56, 0.0)
logloss = (-2277.8901), (cost = 3, inf_term = 1, soc_term = 0), (-1.12, -2.8, -1.12, -1.12, -1.12, 0.0)

In [ ]:
U

In [ ]:
var_to_position = {'x':0,'y':1,'collapse':2}
for i, (cube_for_each_thetas ,unique_thetas,best_thetas) in enumerate(all_results):
    print(file_names[i])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['x'], var_to_position['y'],var_to_position['collapse'],file_names[i+5])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['x'], var_to_position['collapse'],var_to_position['y'],file_names[i+5])
    plot_results(cube_for_each_thetas ,unique_thetas, var_to_position['collapse'], var_to_position['y'],var_to_position['x'],file_names[i+5])

In [ ]:
# best coordinate for UK is 4,0,2,2,5,5,1,1,3
# best coordinate for US is 3,0,3,3,3,5,2,1,0